<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузим-и-изучим-данные" data-toc-modified-id="Загрузим-и-изучим-данные-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузим и изучим данные</a></span></li><li><span><a href="#Лемматизируем-и-очистим-текст" data-toc-modified-id="Лемматизируем-и-очистим-текст-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизируем и очистим текст</a></span></li><li><span><a href="#Сделаем-мешки-характерных-слов" data-toc-modified-id="Сделаем-мешки-характерных-слов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Сделаем мешки характерных слов</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Разделим-данные-на-обучающую,-тестовую-и-валидационную-выборку" data-toc-modified-id="Разделим-данные-на-обучающую,-тестовую-и-валидационную-выборку-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделим данные на обучающую, тестовую и валидационную выборку</a></span></li><li><span><a href="#Найдем-стоп-слова-и-переведем-в-векторы" data-toc-modified-id="Найдем-стоп-слова-и-переведем-в-векторы-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Найдем стоп слова и переведем в векторы</a></span></li><li><span><a href="#Обучим-модель-LogisticRegressor" data-toc-modified-id="Обучим-модель-LogisticRegressor-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучим модель LogisticRegressor</a></span></li><li><span><a href="#Обучим-модель-RandomForestClassifier" data-toc-modified-id="Обучим-модель-RandomForestClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Обучим модель RandomForestClassifier</a></span></li><li><span><a href="#Обучим-модель-DecisionTreeClassifier" data-toc-modified-id="Обучим-модель-DecisionTreeClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Обучим модель DecisionTreeClassifier</a></span></li><li><span><a href="#Проверим-модель-LogisticRegression-на-тестовой-выборке" data-toc-modified-id="Проверим-модель-LogisticRegression-на-тестовой-выборке-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Проверим модель LogisticRegression на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузим и изучим данные

In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import nltk
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [3]:
display(df.head())
print('-'*28)
print(df.describe())
print('-'*28)
print(df.info())
print('-'*28)
print('Количество пропусков:', df.isna().sum())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


----------------------------
               toxic
count  159292.000000
mean        0.101612
std         0.302139
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
----------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None
----------------------------
Количество пропусков: text     0
toxic    0
dtype: int64


In [4]:
df.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Данные загружены корректно, но у целевого признака пристутствует явный дисбаланс классов.

### Лемматизируем и очистим текст

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def get_wordnet_pos(text):   
    tag = nltk.pos_tag([text])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
def clear(text):
    reg = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = reg.split() 
    lemm = [] 
    for i in range(len(clear)):
        lemm.append(lemmatizer.lemmatize(clear[i], get_wordnet_pos(clear[i])))
    return " ".join(lemm)

In [11]:
%%time
df['lemmatize'] = df['text'].apply(clear)

CPU times: user 16min 43s, sys: 1min 40s, total: 18min 24s
Wall time: 18min 25s


In [12]:
df['lemmatize'] = df['lemmatize'].str.lower() #приветдем лемматезированный и  очищенный текст к нижнему регистру

In [13]:
df.head() #проверим результат

,text,toxic,lemmatize
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Создали колонку с лемматизированным и очищенным текстом, а так же привели к нижему регистру

### Сделаем мешки характерных слов

In [14]:
def words_bag(corpus):
    stop_words = set(nltk_stopwords.words('english'))
    count_vect = CountVectorizer(max_features=20, stop_words=stop_words) 
    bow = count_vect.fit_transform(corpus)
    
    print(count_vect.get_feature_names())
    
#сделаем функцию для определения характерных слов

In [15]:
toxic_bag = df[df['toxic'] == 1] 
toxic_bag.head()

,text,toxic,lemmatize
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what be it talk what be it an exclusive gr...
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye don t look come or think of comming back t...
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you be gay or antisemmitian archangel white ti...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,fuck your filthy mother in the ass dry


In [16]:
print(words_bag(list(toxic_bag['lemmatize'])))

['article', 'bitch', 'block', 'die', 'faggot', 'fat', 'fuck', 'gay', 'get', 'go', 'hate', 'know', 'like', 'make', 'nigger', 'page', 'people', 'shit', 'suck', 'wikipedia']
None


In [17]:
no_toxic_bag = df[df['toxic'] == 0] 
no_toxic_bag.head()

,text,toxic,lemmatize
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [18]:
print(words_bag(list(no_toxic_bag['lemmatize'])))

['add', 'also', 'article', 'edit', 'get', 'know', 'like', 'make', 'one', 'page', 'please', 'say', 'see', 'source', 'talk', 'think', 'time', 'use', 'wikipedia', 'would']
None


Определили характерные слов для токсичных и не токсичных комментариев

## Обучение

### Разделим данные на обучающую, тестовую и валидационную выборку

In [19]:
target = df['toxic']
features = df['lemmatize']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=42, stratify=target) 
# stratify=target, чтобы сохранить соотношение классов

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.50, random_state=42, stratify=target_test)
# stratify=target, чтобы сохранить соотношение классов

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(95575,)
(95575,)
(31858,)
(31858,)
(31859,)
(31859,)


In [20]:
print(target_train.value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
print(target_valid.value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
print(target_test.value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
# проверим выборки на дисбаланс классов

0    89.8%
1    10.2%
Name: toxic, dtype: object
0    89.8%
1    10.2%
Name: toxic, dtype: object
0    89.8%
1    10.2%
Name: toxic, dtype: object


### Найдем стоп слова и переведем в векторы

In [21]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [22]:
train_tf_idf = count_tf_idf.fit_transform(features_train)
valid_tf_idf = count_tf_idf.transform(features_valid)
test_tf_idf = count_tf_idf.transform(features_test)

In [23]:
print("train:", train_tf_idf.shape)
print("valid:", valid_tf_idf.shape)
print("test:", test_tf_idf.shape)
# Проверим размер матрицы

train: (95575, 117858)
valid: (31858, 117858)
test: (31859, 117858)


Данные готовы к обучению

### Обучим модель LogisticRegressor

In [24]:
%%time

for c in range(1, 10):
    model_lr = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000, solver='lbfgs', C=c)
    model_lr.fit(train_tf_idf, target_train)
    predicted_valid_lr = model_lr.predict(valid_tf_idf)
    print ("Значение С:", c)
    print("F1_score:", f1_score(target_valid, predicted_valid_lr))
    print('-'*28)
print()

Значение С: 1
F1_score: 0.7420094403236682
----------------------------
Значение С: 2
F1_score: 0.7508232711306256
----------------------------
Значение С: 3
F1_score: 0.7525687309080812
----------------------------
Значение С: 4
F1_score: 0.756476683937824
----------------------------
Значение С: 5
F1_score: 0.7546107278614669
----------------------------
Значение С: 6
F1_score: 0.7544604927782498
----------------------------
Значение С: 7
F1_score: 0.7541915316851379
----------------------------
Значение С: 8
F1_score: 0.7553525549528975
----------------------------
Значение С: 9
F1_score: 0.7553648068669527
----------------------------

CPU times: user 3min 40s, sys: 8min 23s, total: 12min 4s
Wall time: 12min 5s


Модель показала хороший результат, у параметра С=9 F1_score: 0.7659

### Обучим модель RandomForestClassifier

In [25]:
%%time

for depth in range(2, 20, 2):
    model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth = depth)
    model_rf.fit(train_tf_idf, target_train)
    predicted_valid_rf = model_rf.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, predicted_valid_rf))    
    print('-'*28)
print()

Глубина: 2
F1_score: 0.2912723449001052
----------------------------
Глубина: 4
F1_score: 0.2980759009248432
----------------------------
Глубина: 6
F1_score: 0.32992116719614073
----------------------------
Глубина: 8
F1_score: 0.3400133196100987
----------------------------
Глубина: 10
F1_score: 0.3507025392224813
----------------------------
Глубина: 12
F1_score: 0.3719932998324958
----------------------------
Глубина: 14
F1_score: 0.3792697290930507
----------------------------
Глубина: 16
F1_score: 0.38683821978485555
----------------------------
Глубина: 18
F1_score: 0.3907203907203907
----------------------------

CPU times: user 38 s, sys: 270 ms, total: 38.2 s
Wall time: 38.3 s


Лучший показатель у глубины 16 - F1_score: 0.3927

In [26]:
%%time

for est in [100, 200, 300]:
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=16, class_weight='balanced')
    model_rf.fit(train_tf_idf, target_train)
    predicted_valid_rf = model_rf.predict(valid_tf_idf)
    print('Количество деревьев:', est)
    print("F1_score:", f1_score(target_valid, predicted_valid_rf))    
    print('-'*28)
print()

Количество деревьев: 100
F1_score: 0.38437478279001874
----------------------------
Количество деревьев: 200
F1_score: 0.37643028434395337
----------------------------
Количество деревьев: 300
F1_score: 0.3830142707970763
----------------------------

CPU times: user 38.6 s, sys: 240 ms, total: 38.8 s
Wall time: 38.9 s


Лучший показатель с количеством деревьев 200 - F1_score: 0.3788

### Обучим модель DecisionTreeClassifier

In [27]:
for depth in range(2, 20, 2):
    model_dt = DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=depth)
    model_dt.fit(train_tf_idf, target_train)
    predicted_valid_dt = model_dt.predict(valid_tf_idf)
    print('Глубина:', depth)
    print("F1_score:", f1_score(target_valid, predicted_valid_dt))    
    print('-'*28)
print()

Глубина: 2
F1_score: 0.3711648790222
----------------------------
Глубина: 4
F1_score: 0.4691075514874141
----------------------------
Глубина: 6
F1_score: 0.5043821209465382
----------------------------
Глубина: 8
F1_score: 0.5476987447698745
----------------------------
Глубина: 10
F1_score: 0.5756097560975609
----------------------------
Глубина: 12
F1_score: 0.5755395683453237
----------------------------
Глубина: 14
F1_score: 0.5896088733216579
----------------------------
Глубина: 16
F1_score: 0.5998093422306959
----------------------------
Глубина: 18
F1_score: 0.5935973955507325
----------------------------



Лучший показатель у глубины 18 - F1_score: 0.5891

### Проверим модель LogisticRegression на тестовой выборке

In [28]:
%%time

model_lr = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000, solver='lbfgs', C=9)
model_lr.fit(train_tf_idf, target_train)
predicted_test_lr = model_lr.predict(test_tf_idf)
print("F1_score:", f1_score(target_test, predicted_test_lr))

F1_score: 0.7565293093441673
CPU times: user 32 s, sys: 1min 11s, total: 1min 43s
Wall time: 1min 43s


## Выводы

In [29]:
table = [['LogisticRegression', 0.76],
         ['DecisionTreeClassifier', 0.58],
         ['RandomForestClassifier', 0.37]]
table_result = pd.DataFrame(table, columns=['Модель','F1_score'])
table_result = table_result.set_index('Модель')
table_result.index.names = [None]
table_result

,F1_score
LogisticRegression,0.76
DecisionTreeClassifier,0.58
RandomForestClassifier,0.37


# Общий вывод

* Загрузили и изучили данные
* Лемматизировали и очистили текст
* Сделали мешки характерных слов
* Разделили данные на обучающую, тестовую и валидационную выборку
* Нашли стоп слова и перевели в векторы
* Обучили модель LogisticRegressor:
    * Модель показала хороший результат F1_score: 0.7659
* Обучили модель RandomForestClassifier:
    * F1_score: 0.3927
* Обучили модель DecisionTreeClassifier:
    * F1_score: 0.5891
* Проверили модель LogisticRegression на тестовой выборке:
    * F1_score: 0.7631
